In [256]:
from os import listdir
import csv
import os
import numpy as np
import pandas as pd
from scipy.misc import imresize, imread, imshow
import skimage
from skimage import io
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt

In [278]:
def load_img(img_filename):
    """
    Load image from the filename. Default is to load in color if
    possible.
    Args:
        img_name (string): string of the image name, relative to
            the image directory.
    Returns:
        np array of float32: an image as a numpy array of float32
    """
    img = skimage.img_as_float(imread(
        img_filename,mode='RGB')).astype(np.float32)
    if img.ndim == 2:
        img = img[:, :, np.newaxis]
    elif img.shape[2] == 4:
        img = img[:, :, :3]
    
    '''Subtract channel vise mean'''
    ch_mean = np.load('preprocessing/ch_mean.npy')
    img = img - ch_mean
    return img

In [265]:
class DictIterator():
    '''
    Class for iterating over a pandas data frame with filenames and addresses

    TODO
    add comments
    '''
    def __init__(self,csv_location = None,
                 batch_size=32,
                 shuffle=False,
                 target_size=(64,64)):
        
        if csv_location is not None:
            self.df = pd.read_csv(csv_location)
        self.N = self.df.shape[0]
        print(self.N)
        self.batch_size = batch_size
        self.total_batches_seen = 0
        self.index_gen = self._idx_gen(self.N,batch_size,shuffle)
        self.target_size = target_size
        self.img_set = None
        
    def _idx_gen(self,N,batch_size=32,shuffle=False):
        batch_index = 0
        while 1:
            if batch_index == 0:
                index_array = np.arange(N)
                if shuffle:
                    index_array = np.random.permutation(N)
            current_index = (batch_index * batch_size) % N
            if N >= current_index + batch_size:
                current_batch_size = batch_size
                batch_index += 1
            else:
                current_batch_size = N - current_index
                batch_index = 0
            self.total_batches_seen += 1
            yield (index_array[current_index: current_index + current_batch_size],
                   current_index, current_batch_size)
            
    def batch_gen(self):
        while 1:
            index_array, current_index, current_batch_size = self.index_gen.__next__()
            img = [imresize(load_img(self.df.iloc[i]['filename']),size=self.target_size) for i in index_array]
            img = np.array(img)
            lab = [self.df.iloc[i].iloc[1:].values.astype('float32') for i in index_array]
            lab = np.array(lab)
            print(img.shape)
            print(lab.shape)
            plt.imshow(img[0])
            yield img,lab
    
    def channel_mean(self):
        self.img_set = np.array([load_img(self.df.iloc[i]['filename']) for i in range(self.N)])
        
        

In [266]:
d = DictIterator(csv_location='preprocessing/trainset.csv')
d.channel_mean()

100000


In [271]:
print(d.img_set.shape)
r = np.mean(d.img_set[:,:,:,0],  axis=0)
g = np.mean(d.img_set[:,:,:,1],  axis=0)
b = np.mean(d.img_set[:,:,:,2],  axis=0)



np.save('preprocessing/r.npy',r)
np.save('preprocessing/g.npy',g)
np.save('preprocessing/b.npy',b)



(100000, 64, 64, 3)


In [277]:
ch_mean = np.zeros((64,64,3))
print(ch_mean.shape)
ch_mean[:,:,0] = r
ch_mean[:,:,1] = g
ch_mean[:,:,2] = b
np.save('preprocessing/ch_mean.npy',ch_mean)

(64, 64, 3)


In [240]:
from model_defs import *
from utils import *
import argparse
import pandas as pd
import os
import numpy as np
import csv
from keras.models import model_from_json




            

In [249]:
def try_modelx(csv_location='preprocessing/valset.csv',batch_size=64,mid='m1'):


    #define the batch generator   (validation set)
    val_datagen = CSVGenerator(csv_location=csv_location,
                                 batch_size=batch_size)

    val_generator = val_datagen.batch_gen()

    x = tf.placeholder(tf.float32, shape=(None, 64, 64, 3))

    # load json and create model
    json_file = open('models/'+mid+'.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    model = model_from_json(loaded_model_json)
    # load weights into new model
    model.load_weights("models/"+mid+".h5")
    print("Loaded model from disk")

    d = val_generator.__next__()
    pred = model(x)
    pred
    # evaluate loaded model on test data
    #model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    #val_loss = model.evaluate_generator(
    #   generator = val_generator,
    #   val_samples = val_datagen.get_data_size())

    #print("%s: %.2f%%" % (model.metrics_names[1], val_loss[1]*100))
    #print(val_loss)

    pass

In [250]:
try_modelx()


10000
Loaded model from disk


In [252]:
import os
directory = '/dccstor/dlw/ambrish/examples/adversarial/models/m1'
if not os.path.exists(directory):
    os.makedirs(directory)

In [254]:
mid = 'm1'
epoch = 100
val_loss = [1.32,23.4]
with open('models/'+mid+'/acc_log.csv','a') as resultFile:
    r = np.concatenate([np.asarray([epoch]),val_loss])
    wr = csv.writer(resultFile,lineterminator='\n')
    
    wr.writerows([r])
    wr.writerow(['f','r','r'])

In [255]:
from os import listdir
import csv
import os
import numpy as np
import pandas as pd
from scipy.misc import imresize, imread, imshow
import skimage
from skimage import io
import matplotlib
import matplotlib.pyplot as plt
import argparse
from pathlib import Path

class tinyImageNet(object):

    def __init__(self,fpath=None):
        self.fpath = fpath
        self.classes = None

    def make_train_csv(self):
        '''
        for tiny ImageNet folder structure (train)
        '''
        self.classes = listdir(os.path.join(self.fpath,'train'))
        num_classes = len(self.classes)
        traincsv_file = Path('trainset.csv')
        if traincsv_file.is_file():
            pass
        else:
            with open('trainset.csv', 'wt') as csvfile:
                csvwriter = csv.writer(csvfile, delimiter=',')
                csvwriter.writerow(['filename']+self.classes) #csv header
    
                c_idx = 0
                for class_ in self.classes:
                    class_path = os.path.join(self.fpath,'train',class_,'images')
                    images_ = listdir(class_path)
                    for image_ in images_:
    
                        file_path = os.path.join(class_path,image_)
    
                        label = np.zeros(num_classes)
                        label[c_idx] = 1
    
                        csvwriter.writerow([file_path] + list(map(str, label)))
                    c_idx+=1

    def make_val_csv(self):
        '''
        for tiny ImageNet folder structure (val)
        '''
        valcsv_file = Path('val.csv')
        if valcsv_file.is_file():
            pass
        else:
            with open(os.path.join(self.fpath,'val/val_annotations.txt'), 'rt') as csvfile:
                
                reader = csv.reader(csvfile, delimiter='\t')
                
                with open('valset.csv', 'wt') as csvfile:
                    
                    csvwriter = csv.writer(csvfile, delimiter=',')
                    csvwriter.writerow(['filename']+self.classes)
                    
                    for row in reader:

                        file_path = os.path.join(self.fpath,'val','images',row[0])
                        
                        label = np.zeros(len(self.classes))
                        label[self.classes.index(row[1])] = 1
                        
                        csvwriter.writerow([file_path] + list(map(str, label)))            
                
    def make_csvs(self):
        self.make_train_csv()
        self.make_val_csv()